### NLP (Natural Language Processing)-machine learning Classification  3( N-gram vectors)

Dans l'article précedent, nous avons décomposé le workflow de classification de texte en plusieurs étapes. pour répondre à deux questions clés:


* Quel algorithme ou modèle d'apprentissage devrions-nous utiliser?

* Comment préparer les données pour apprendre efficacement la relation entre le texte et l'étiquette?


Pour chaque étape, nous avons proposé une approche personnalisée basée sur les caractéristiques spécifique de votre jeu de données. En particulier, en utilisant le rapport du nombre d'échantillons au nombre de mots par échantillon, nous avons suggéré deux approche .

La réponse à la deuxième question dépend de la réponse à la première question; la façon dont nous pré-traitons les données à introduire dans un modèle dépendra du modèle que nous choisirons. Les modèles peuvent être classés en deux grandes catégories:

* ceux qui utilisent le texte comme une séquence des mots (modèles de séquence) 

* ceux qui ne voient que le texte comme des «sac de mots>>  (modèles à n-grammes). 

Les types de modèles de séquence comprennent les réseaux de neurones convolutifs (CNN), les réseaux de neurones récurrents (RNN) et leurs variations.

Les types de modèles à n-grammes incluent la régression logistique , les perceptrons multicouches(MLP), les arbres boostés par le gradient et les machines à vecteurs de support.... .


Lorsque la valeur de ce rapport est petite (<1500), les modèles  qui prennent les n-grammes en entrée  fonctionnent mieux ou au moins aussi bien que les modèles de séquence.

Les modèles  qui prennent les n-grammes en entrée sont simples à définir et à comprendre, et ils prennent moins de temps de calcul que les modèles de séquence. Lorsque la valeur de ce rapport est grande (> = 1500), utilisez un modèle de séquence . 


Dans cet article , nous travaillerons à la construction, et à l'évaluation des modèles à n-grammes. 

Certains des algorithmes de classification à n-grammes les plus largement utilisés sont la régression logistique,les perceptrons multicouches(MLP),  Naive Bayes, le plus proche voisin K et les méthodes d'arbre.... ensembles

## Étape 4: Créez, formez et évaluez votre modèle

À l' étape 3 , nous avons choisi d'utiliser un modèle à n-grammes ou un modèle de séquence.
Maintenant, il est temps d'écrire notre algorithme de classification et de le former.

#### Construire un modèle n-gram

Nous nous référons aux modèles qui traitent les tokens indépendamment (sans tenir compte de l'ordre des mots) en tant que modèles à n-grammes. Les perceptrons multicouches  (y compris la régression logistique ), Naive Bayes, le plus proche voisin K et les méthodes d'arbre.... , entrent tous dans cette catégorie

Découvrons chacun d'eux ensembles

On recupere tout d'abord les données, et de créer deux DataFrame Pandas, un pour l'apprentissage, l'autre pour la validation.

In [5]:
import sklearn.model_selection as sms
import pandas as pd

In [6]:
def split_dataset(input_path, nb_line, tauxValid):
    data_all = pd.read_csv(input_path,sep=",", nrows=nb_line)
    data_all = data_all.fillna("")
    data_train, data_valid = sms.train_test_split(data_all, test_size=tauxValid, random_state=47, shuffle=True)
    return data_train, data_valid

In [7]:
input_path = "data/cdiscount_train.csv.zip"
nb_line=50000 # part totale extraite du fichier initial ici déjà réduit
tauxValid = 0.2
data_train, data_valid = split_dataset(input_path, nb_line, tauxValid)
data_train.reset_index(inplace = True)
data_valid.reset_index(inplace = True)

N_train = data_train.shape[0]
N_valid = data_valid.shape[0]
print("Train set : %d elements, Validation set : %d elements" %(N_train, N_valid))

Train set : 40000 elements, Validation set : 10000 elements


Vectoriser en utilisant l'encodage tf-idf,
Sélectionnez uniquement les 20 000 premières fonctionnalités du vecteur de jetons en supprimant les jetons qui apparaissent moins de 2 fois et en utilisant f_classif pour calculer l'importance des fonctionnalités.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from spacy.lang.fr import French

nlp = French()

def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower() for word in mytokens if
                not word.is_punct and not word.like_num and word.text != 'n']

    # Removing stop words
    #mytokens = [word for word in mytokens if word not in stop_words]

    # Remove accentuated char for any unicode symbol
    mytokens = [strip_accents_ascii(word) for word in mytokens]

    # return preprocessed list of tokens
    return mytokens

# Vectorization parameters
# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 2)

# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Whether text should be split into word or character n-grams.
# One of 'word', 'char'.
TOKEN_MODE = 'word'

# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

def ngram_vectorize(train_texts, train_labels, val_texts):
    """Vectorizes texts as n-gram vectors.

    1 text = 1 tf-idf vector the length of vocabulary of unigrams + bigrams.

    # Arguments
        train_texts: list, training text strings.
        train_labels: np.ndarray, training labels.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val: vectorized training and validation texts
    """
    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
            'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE,  # Split text into word tokens.
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train = vectorizer.fit_transform(train_texts)

    # Vectorize validation texts.
    x_val = vectorizer.transform(val_texts)

    # Select top 'k' of the vectorized features.
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val

In [13]:
x_train,x_val = ngram_vectorize(data_train["Description"], data_train["Categorie1"], data_valid["Description"])

C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


###### Bagging

Le principe des méthodes de Bagging, et donc en particulier des forêts aléatoires c’est de faire la moyenne des prévisions de plusieurs modèles indépendants pour réduire la variance et donc l’erreur de prévision. Pour construire ces différents modèles, on sélectionne plusieurs échantillons bootstrap, c’est à dire des tirages avec remises.

![](data/bagging.png)

######  Random Forest

Cet algorithme appartient à la famille des agrégations de modèles, c’est en fait un cas particulier de bagging (bootstrap aggregating) appliqué aux arbres de décision de type CART. 
En plus du principe de bagging, les forêts aléatoires ajoutent de l’aléa au niveau des variables. Pour chaque arbre on sélectionne un échantillon bootstrat d’individus et à chaque étape, la construction d’un noeud de l’arbre se fait sur un sous-ensemble de variables tirées aléatoirement.

On se retrouve donc avec plusieurs arbres et donc des prédictions différentes pour chaque individu. Comment obtenir l’estimation finale?

* Dans le cas d’une classification : on choisit la catégorie la plus fréquente
* Dans le cas d’une régression : on fait la moyenne des valeurs prédites

![](data/forest.png)

###### Boosting (AdaBoost)

L’idée de base ressemble à celle du bagging. Plutôt que d’utiliser un seul modèle, nous en utilisons plusieurs que nous agrégeons ensuite pour obtenir un seul résultat. Dans  la construction des modèles, le Boosting travaille de manière séquentielle. Il commence par construire un premier modèle qu’il va évaluer. A partir de cette mesure, chaque individu va être pondéré en fonction de la performance de la prédiction. L’objectif est de donner un poids plus important aux individus pour lesquels la valeur a été mal prédite pour la construction du modèle suivant. Le fait de corriger les poids au fur et à mesure permet de mieux prédire les valeurs difficiles.

![](data/ada.png)

###### gradient boosting

L’algorithme de boosting de gradient (gradient boosting) figure aussi parmi les méthodes de boosting très appréciées. Tout comme AdaBoost, le boosting de gradient travaille par ajout séquentiel de prédicteurs à un ensemble, chacun d’eux corrigeant son prédécesseur. Cependant, au lieu de modier légèrement les poids des observations à chaque itération comme le fait AdaBoost, cette méthode tente d’ajuster un nouveau prédicteur aux erreurs résiduelles du prédicteur précédent.

Cet algorithme utilise le gradient de la fonction de perte pour le calcul des poids des individus lors de la construction de chaque nouveau modèle. Cela ressemble un peu à la descente de gradient pour les réseaux de neurones.


![](data/Gradient_boosting.png)

###### régression logistique
La régression logistique est utilisée couramment pour estimer la proba-bilité qu’une observation appartienne à une classe particulière (p. ex. quelle est la probabilité que cet e-mail soit un pourriel?). Si la probabilité estimée est supérieure à 50%, alors le modèle prédit que l’observation appartient à cette classe (appelée la classe positive, d’étiquette «1»), sinon il prédit qu’elle appartient à l’autre classe (la classe négative, d’étiquette «0»). C’est en fait un classificateur binaire.

Tout comme un modèle de régression linéaire, un modèle de régression logistique calcule une somme pondérée des caractéristiques d’entrée (plus un terme constant), mais au lieu de fournir le résultat directement comme le fait le modèle de régression linéaire, il fournit la logistique du résultat.

Le modèle de régression logistique peut être généralisé de manière à prendre en compte plusieurs classes directement, sans avoir à entraîner plusieurs classificateurs binaires puis à les combiner. C’est ce qu’on appelle la régression softmax, ou régression logistique multinomiale.
Le principe en est simple: étant donné une observation x, le modèle de régression softmax calcule d’abord un score s_k(x) pour chaque classe k, puis estime la probabilité de chaque classe en appliquant aux scores la fonction softmax . La formule permettant de calculer s_k(x) devrait vous sembler familière, car c’est la même que pour calculer la prédiction en régression linéaire.

#### Machines à vecteurs  de support

Une machine à vecteurs de support ou séparateur à vaste marge (SVM) est un modèle d’apprentissage automatique à la fois puissant et polyvalent, capable d’effectuer des classifications linéaires ou non linéaires, des régressions et même des détections de données aberrantes. C’est un des modèles les plus prisés en matière d’apprentissage automatique, et tous ceux qui s’intéressent au Machine Learning devraient en avoir dans leur boîte à outils. Ils sont particulièrement bien adaptés à la classification de jeux de données complexes, de taille réduite ou moyenne.
L'idée de Support Vector Machines est simple: l'algorithme crée une ligne qui sépare les classes, dans le cas par exemple d'un problème de classification. Le but de la ligne est de maximiser la marge entre les points de chaque côté de la ligne dite de décision. L'avantage de ce processus est qu'après la séparation, le modèle peut facilement deviner les classes cibles (étiquettes) pour les nouveaux cas

![](data/svm.png)

###### Naive Bayes Classifier
La classification bayésienne naïve (Naive Bayes classifier), est utilisée depuis longtemps dans l'industrie et le monde universitaire (introduite par Thomas Bayes ), Cependant, cette technique est à l'étude depuis les années 50 pour la catégorisation des textes et des documents. 
Naive Bayes Classifier est un modèle génératif qui est largement utilisé dans la recherche d'informations, qui se base sur le théorème de Bayes. 

![](data/NB.png)

###### réseaux de neurones profond

Les architectures des réseaux neuronaux profonds sont conçues pour apprendre à travers la connexion multiple de couches où chaque couche unique ne reçoit que la connexion de la précédente et fournit des connexions uniquement à la couche suivante dans la partie cachée. L'entrée est une connexion de l'espace d'entité (avec la première couche cachée). Pour les réseaux neuronaux profonds , la couche d'entrée peut être tf-ifd, one-hot encoding etc.., la couche de sortie contient des neurones égaux au nombre de classes pour la classification multi-classes, et un seul neurone pour la classification binaire.

Lorsque nous n'avons que 2 classes (classification binaire), notre modèle devrait produire un seul score de probabilité. Par exemple, la sortie 0.2 pour un échantillon d'entrée donné signifie «20% de confiance que cet échantillon est dans la première classe (classe 1), 80% qu'il est dans la deuxième classe (classe 0)». Pour produire un tel score de probabilité, la fonction d'activation de la dernière couche doit être une fonction sigmoïde et la fonction de perte utilisée pour entraîner le modèle doit être une entropie croisée binaire (binary cross-entropy) .

Lorsqu'il y a plus de 2 classes (classification multi-classes), notre modèle devrait produire un score de probabilité par classe. La somme de ces scores doit être 1. Par exemple, la sortie {0: 0.2, 1: 0.7, 2: 0.1}signifie «20% de confiance que cet échantillon est dans la classe 0, 70% qu'il est dans la classe 1 et 10% qu'il est dans la classe 2.» Pour produire ces scores, la fonction d'activation de la dernière couche doit être softmax et la fonction de perte utilisée pour entraîner le modèle doit être une entropie croisée catégorique(categorical cross-entropy).

![](data/Rn.jpg)
